In [9]:
import pandas as pd
import numpy as np
from copulas.multivariate import GaussianMultivariate, VineCopula
import warnings
warnings.filterwarnings("ignore")

# Load and clean data
data = pd.read_excel("synthetic_data.xlsx").dropna()
data = data.loc[:, data.nunique() > 1]
data = data.loc[:, data.std() > 1e-6]

# Copula models
copula_models = {
    'GaussianMultivariate': GaussianMultivariate(),
    'VineCopula': VineCopula('center')
}

results = []

for name, model in copula_models.items():
    try:
        model.fit(data)
        log_lik = np.sum(np.log(model.probability_density(data)))
        d = data.shape[1] if name == "GaussianMultivariate" else len(model.get_parameters())
        k = int(d * (d - 1) / 2) if name == "GaussianMultivariate" else d
        aic = -2 * log_lik + 2 * k
        results.append((name, log_lik, k, aic))
    except Exception as e:
        print(f"❌ Error with {name}: {e}")

        # Special handling for VineCopula: find the bad column
        if name == "VineCopula":
            print("🔍 Trying to identify problematic feature...")
            for col in data.columns:
                try:
                    model.fit(data.drop(columns=[col]))
                    print(f"✅ VineCopula fits after dropping: {col}")
                except:
                    continue
        results.append((name, None, None, float('inf')))

# Final results
results_df = pd.DataFrame(results, columns=["Copula", "Log-Likelihood", "Num Params", "AIC"])
results_df = results_df.sort_values("AIC")

print("\n📊 Multivariate Copula Comparison:")
print(results_df.to_string(index=False))
print(f"\n✅ Best Copula Based on AIC: {results_df.iloc[0]['Copula']}")


❌ Error with VineCopula: Constant column.
🔍 Trying to identify problematic feature...
✅ VineCopula fits after dropping: Voltage (V)
✅ VineCopula fits after dropping: Resistance (Ohm)

📊 Multivariate Copula Comparison:
              Copula  Log-Likelihood  Num Params           AIC
GaussianMultivariate    22780.425038        78.0 -4.540485e+04
          VineCopula             NaN         NaN           inf

✅ Best Copula Based on AIC: GaussianMultivariate


In [1]:
import pandas as pd
import numpy as np
from copulas.multivariate import GaussianMultivariate, VineCopula
import warnings
warnings.filterwarnings("ignore")

# Load your Excel file
data = pd.read_excel("synthetic_data.xlsx").dropna()

# Define multivariate copula models
copula_models = {
    'GaussianMultivariate': GaussianMultivariate(),
    'VineCopula': VineCopula('center')  # use C-vine structure
}

results = []

# Fit and evaluate each model
for name, model in copula_models.items():
    try:
        model.fit(data)
        log_lik = np.sum(np.log(model.probability_density(data)))
        k = len(model.get_parameters())  # number of free parameters
        aic = -2 * log_lik + 2 * k
        results.append((name, log_lik, k, aic))
    except Exception as e:
        print(f"Error with {name}: {e}")
        results.append((name, None, None, float('inf')))

# Display results
results_df = pd.DataFrame(results, columns=["Copula", "Log-Likelihood", "Num Params", "AIC"])
results_df = results_df.sort_values("AIC")

print("\nMultivariate Copula Comparison:")
print(results_df.to_string(index=False))

# Best model
best_model = results_df.iloc[0]["Copula"]
print(f"\n✅ Best Copula Based on AIC: {best_model}")


Error with GaussianMultivariate: 'GaussianMultivariate' object has no attribute 'get_parameters'
Error with VineCopula: Constant column.

Multivariate Copula Comparison:
              Copula Log-Likelihood Num Params  AIC
GaussianMultivariate           None       None  inf
          VineCopula           None       None  inf

✅ Best Copula Based on AIC: GaussianMultivariate


In [4]:
import pandas as pd

# Load the data
file_path = "synthetic_data.xlsx"
data = pd.read_excel(file_path)

# Identify columns with a single unique value
constant_columns = data.columns[data.nunique() <= 1].tolist()
print(constant_columns)


[]


In [5]:
import pandas as pd
import numpy as np
from copulas.multivariate import GaussianMultivariate, VineCopula
import warnings
warnings.filterwarnings("ignore")

# Step 1: Load the data
data = pd.read_excel("synthetic_data.xlsx").dropna()

# Step 2: Drop constant or near-constant columns
data = data.loc[:, data.nunique() > 1]         # drop exactly constant
data = data.loc[:, data.std() > 1e-6]          # drop near-constant (tiny std)

# Step 3: Define copula models
copula_models = {
    'GaussianMultivariate': GaussianMultivariate(),
    'VineCopula': VineCopula('center')
}

# Step 4: Fit and evaluate models
results = []
for name, model in copula_models.items():
    try:
        model.fit(data)
        log_lik = np.sum(np.log(model.probability_density(data)))

        # Estimate number of parameters
        if name == "GaussianMultivariate":
            d = data.shape[1]
            k = int(d * (d - 1) / 2)  # number of correlations in correlation matrix
        else:
            k = len(model.get_parameters())

        aic = -2 * log_lik + 2 * k
        results.append((name, log_lik, k, aic))
    except Exception as e:
        print(f"❌ Error with {name}: {e}")
        results.append((name, None, None, float('inf')))

# Step 5: Display results
results_df = pd.DataFrame(results, columns=["Copula", "Log-Likelihood", "Num Params", "AIC"])
results_df = results_df.sort_values("AIC")

print("\n📊 Multivariate Copula Comparison:")
print(results_df.to_string(index=False))

# Step 6: Show best model
best_model = results_df.iloc[0]["Copula"]
print(f"\n✅ Best Copula Based on AIC: {best_model}")


❌ Error with VineCopula: Constant column.

📊 Multivariate Copula Comparison:
              Copula  Log-Likelihood  Num Params           AIC
GaussianMultivariate    22780.425038        78.0 -4.540485e+04
          VineCopula             NaN         NaN           inf

✅ Best Copula Based on AIC: GaussianMultivariate


In [6]:
import pandas as pd
import numpy as np
from copulas.multivariate import GaussianMultivariate, VineCopula
import warnings
warnings.filterwarnings("ignore")

# Load your Excel file
data = pd.read_excel("synthetic_data.xlsx").dropna()

# Strict filtering for VineCopula
data = data.loc[:, (data.std() > 1e-3) & (data.nunique() > 5)]

# Define copula models
copula_models = {
    'GaussianMultivariate': GaussianMultivariate(),
    'VineCopula': VineCopula('center')  # C-vine structure
}

results = []

# Fit and evaluate
for name, model in copula_models.items():
    try:
        model.fit(data)
        log_lik = np.sum(np.log(model.probability_density(data)))

        # Estimate number of parameters
        if name == "GaussianMultivariate":
            d = data.shape[1]
            k = int(d * (d - 1) / 2)
        else:
            k = len(model.get_parameters())

        aic = -2 * log_lik + 2 * k
        results.append((name, log_lik, k, aic))
    except Exception as e:
        print(f"❌ Error with {name}: {e}")
        results.append((name, None, None, float('inf')))

# Create results DataFrame
results_df = pd.DataFrame(results, columns=["Copula", "Log-Likelihood", "Num Params", "AIC"])
results_df = results_df.sort_values(by="AIC")

print("\n📊 Copula Comparison (Filtered Data):")
print(results_df.to_string(index=False))

print(f"\n✅ Best Copula: {results_df.iloc[0]['Copula']}")


❌ Error with VineCopula: Constant column.

📊 Copula Comparison (Filtered Data):
              Copula  Log-Likelihood  Num Params         AIC
GaussianMultivariate     31439.40975        55.0 -62768.8195
          VineCopula             NaN         NaN         inf

✅ Best Copula: GaussianMultivariate


In [8]:
low_unique_cols = data.columns[data.nunique() <= 5]
if len(low_unique_cols) > 0:
    print("⚠️ Columns with ≤ 5 unique values:")
    for col in low_unique_cols:
        print(f"• {col} → {data[col].nunique()} unique values: {data[col].unique()}")
else:
    print("✅ All columns have enough unique values.")


✅ All columns have enough unique values.
